# GJAM  methods
### Author: Alyssa Willson

This document will describe the  methods for running the Generalized Joint Attribute Model (GJAM) to analyze the relationship between the environment and the plant community directly before European settlement in the Midwest, US. I will first describe the data, briefly describe the model, and then discuss its implementation.

## Objective

The objective of the methods described below is to understand the drivers of tree community distribution in the Midwest, U.S. prior to widespread European settlement. The vegetation community can be influenced by both biotic and abiotic drivers. Abiotic drivers include climate, soil conditions, and topography. Biotic drivers include inter- and intraspecific competition and facilitation. We reconstructed the vegetation community and a suite of abiotic drivers for the period just prior to European settlement, corresponding to the mid-19<sup>th</sup> century for Indiana and Illinois, U.S. We used the Generalized Joint Attribute Model (GJAM) to evaluate the relative contributions of abiotic and biotic factors in driving the distribution of the vegetation communities within this region.

## Data

***The Public Land Survey***

The Public Land Survey is a dataset that was collected by the U.S. Bureau of Land Management in the early and late 19<u>th</u> century. The survey was conducted in order to plat the land for sale. In so doing, surveyors used trees to delineate distance at regular intervals (called *corners*) across much of the contiguous U.S. and recorded the trees' taxon. From these regular spaced records of tree taxon identity, we collated a digitized record of the trees identified in the Public Land Survey in Illinois and Indiana, representing the plant community across this region before large-scale European settlement.

***Vegetation community reconstruction***

We used the Public Land Survey record to identify the presence of 14 plant taxa across our study region. We defined our study region as a set of discrete "management areas" that represent the breadth of environmental conditions and ecological communities observed across Indiana and Illinois, and also represent areas of high current management priority. We grouped taxa recorded in the Public Land Survey into 14 taxa that both represent the plant communities of the region and reduce the number of taxa with few observations in our dataset (Table 1). We further grouped the taxa into three biomes: prairie, savanna, and forest. The category of "no tree" is indicative of prairie because this suggests that the PLS surveyor did not encouter any tree within a considerable distance of the corner, consistent with a tree-less prairie. The oak and hickory taxa are indicative of savanna when no trees of a different taxa are observed because oaks and hickories are the dominant trees in the oak savannas of the Midwest region. All other tree taxa are assumed to be indicative of forest of any kind (Table 1).

|Biome | Taxon | PLS Taxon|
|------|-------|----------|
|Prairie|No tree|No tree|
|Savanna|Oak|Oak|
|Savanna|Hickory|Hickory|
|Forest|Elm|Elm|
|Forest|Ash|Ash|
|Forest|Maple|Maple|
|Forest|Basswood|Basswood|
|Forest|Walnut|Walnut|
|Forest|Ironwood|Ironwood|
|Forest|Beech|Beech|
|Forest|Dogwood|Dogwood|
|Forest|Poplar/Tulip Poplar|Poplar|
|Forest|Poplar/Tulip Poplar|Tulip Poplar|
|Forest|Poplar/Tulip Popoar|Poplar/Tulip Poplar|
|Forest|Black Gum/Sweet Gum|Black Gum|
|Forest|Black Gum/Sweet Gum|Sweet Gum|
|Forest|Black Gum/Sweet Gum|Black Gum/Sweet Gum|
|Forest|Other Conifer|Bald Cypress|
|Forest|Other Conifer|Pine|
|Forest|Other Conifer|Tamrack|
|Forest|Other Hardwood|Birch|
|Forest|Other Hardwood|Locust|
|Forest|Other Hardwood|Willow|
|Forest|Other Hardwood|Cherry|
|Forest|Other Hardwood|Sycamore|
|Forest|Other Hardwood|Buckeye|
|Forest|Other Hardwood|Hackberry|
|Forest|Other Hardwood|Mulberry|

Table 1. Taxa in Public Land Survey. **Biome**: The biome indicated by the presence of each taxon. For example, if no tree is present, then the data indicate that the location is a prairie. **Taxon**: The reduced list of taxa used in our analysis to reduce the number of low-frequency taxa in our dataset. **PLS Taxon**: The taxon recorded in the Public Land Survey (PLS).

***Environmental drivers***

We reconstructed a suite of environmental drivers. Broadly, the drivers fall into three categories: climate drivers, soil characteristics, and topographic conditions. The specific variables within each category are available in Table 2.

|Variable Name|Variable Type|Method|
|-------------|-------------|------|
|Mean Slope|Topographic|Point-level extraction from USGS Digital Elevation Model map|
|Mean Aspect|Topographic|Point-level extraction from USGS Digital Elevation Model map|
|Mean CaCO<sub>3</sub> Concentration|Soil|Point-level extraction from USDA's gSSURGO soil data product|
|Soil Cation Exchange Capacity|Soil|Point-level extraction from USDA's gSSURGO soil data product|
|Soil % Clay|Soil|Point-level extraction from USDA's gSSURGO soil data product|
|Soil % Sand|Soil|Point-level extraction from USDA's gSSURGO soil data product|
|Available Water Content|Soil|Point-level extraction from USDA's gSSURGO soil data product|
|Saga Wetness Index|Topographic|Calculation using point-level slope, aspect, and the RSAGA R package|
|Hydric Soil Presence|Soil|Point-level extraction from USDA's gSSURGO soil data product|
|Floodplain Presence|Soil|Point-level extraction from USDA's gSSURGO soil data product|
|Mean Annual Precipitation 1895-1925|Climate|Point-level extraction from PRISM reconstruction|
|Mean Annual Temperature 1895-1925|Climate|Point-level extraction from PRISM reconstruction|

Table 2. Environmental drivers. Variable type correspond to the three broad categories defined in the methods. Method describes how the drivers were obtained. More details available in Supplement.

## Process Model

***Model description**

We used the Generalized Joint Attribute Model (GJAM; Clark et al. 2017). GJAM is a hierarchical Bayesian model develoepd for modeling complex ecological data. Briefly, the model estimates the relationship between any number of drivers and response variables using a series of linear models. Drivers can be continuous or discrete, including binary. Response variables can be continuous or discrete, encompassing the most popular types of ecological data, such as presence/absence, mark/recapture, and discrete abundance. The response variables can also vary in their type and scale. The flexibility of the model is accomplished by jointly mapping the response variables to a latent, continuous, multivariate variable. The mapping involves using a variable called the measurement "effort," which can be thought of as the level of precision or confidence one has in each observation (or each repeat measurement) in the state variable.

The GJAM modeling procedure is a suitable tool for the analysis conducted herein for several reasons. First, the modeling of a multivariate latent state allows us to explicitly quantify the covariance between the response variables (*i.e.*, the presence or absence of each of our tree taxa and the "no tree" category). Second, the implementation of "effort" offers a means for explicitly including the certainty in the response data in the analysis. This is particularly important in our case because the collection method of our data (*i.e.*, identifying one tree near each measurement point; see Paciorek et al. for more details), which has variable uncertainty depending on both the location and the taxon. Third, GJAM was developed specifically to handle zero-inflated data, which is common in ecological data. Our data contain mostly zeros for most taxa; it is likely that some of these zeros are not indicative of a species being absent, but rather that a tree of a different taxon was recorded in the PLS dataset in its place. This scenario represents true zero-inflation, wherein zeros represent multiple processes in the data (*i.e.*, true absence and false negatives), making accounting for zero-inflation in our process model crucial.

***Model implementation***

Before fitting the model, we withheld one third of the management areas in our dataset for validation. The rest of this section will focus on the two-thirds of the data we used for training the model. The training data sets consists of 78,224 observations of 14 tree taxa and the "no tree" category (15 total response variables) and 78,224 observations of 14 environmental drivers (12 environmental drivers from Table 2 and latitude and longitude). We also defined a matrix of effort with 78,224 observations for each of thev 15 taxa in the response variable. Effort was defined as the inverse of the distance from eah corner measured in chains. During surveying, trees suitable for demarcating a given corner for platting the land may not be available directly at the location of the corner. For this reason, surveyors may travel considerable distance to use a tree to identify the corner. We consider that trees located at greater distance from the corner bear less information about the forest community at a given corner because we have less information about the trees direclty around the corner. We take the inverse of the distance in defining effort because the inverse represents the certainty, or precision, of the measurement, consistent with the definition of effort. Each observation corresponds to a different corner.

We used the `gjam` R package (Clark et al. 2017) following the published protocols for implementation. Specifically, we specified three Markov Chain Monte Carlo chains, each with a burn-in period of 250 iterations and with 1000 total iterations. We specified three random effects: management area, latitude, and longitude. We did this because to our knowledge, GJAM cannot account for spatial covariance and we understand that forest community composition is strongly correlated in space. Finally, we define the formula as a simple linear regression with no interactions:

$$\boldsymbol{Y} = \boldsymbol{\beta X}$$

where $\boldsymbol{Y}$ is the presence or absence of each taxon at each corner, $\boldsymbol{X}$ is a matrix of the environmental drivers (Table 2) at each corner, and $\boldsymbol{\beta}$ represents the coefficients of the model. Bear in mind that this formula only represents one portion of the hierarchical model in GJAM. Covariance between the response variables is accomplished in a different portion of the model. More details about the model and its implementation in various settings in ecology can be found in Clark et al. 2017.

***Response variable dimension reduction***

Because of the large size of our dataset, the complexity of the linear model we specified, and the complexity of the underlying GJAM model, reducing the dimensionality of our data became imperative. We began by identifying the number of times each taxon in our dataset was present in our full dataset (Figure 1, left). Given that several taxa had fewer than 1,000 observations out of over 75,000 total observations, we decided to group several rare taxa into one of two categories: other hardwood or other conifer. Specifically, we grouped Bald Cypress, Pine, and Tamarack into the "other conifer" category and we grouped Birch, Locust, Willow, Cherry, Sycamore, Buckeye, Hackberry, and Mulberry into "other hardwood." It is important to note that the "other hardwood" taxon already existed in our dataset and therefore represents more taxa than those listed above. Additionally, we grouped Poplar and Tulip Poplar into the taxon Poplar/Tulip Poplar. While the taxa of Poplar and Tulip Poplar are taxonomically and ecologically distinct, they are poorly distinguished in the PLS record and cannot be adequately distinguished in our dataset. Finally, we combined Black Gum and Sweet Gum for a similar reason. This procedure reduced the number of response variables from 29 (Figure 1, left) to 15 (Figure 1, right). For our presence/absence data, we combined variables by considering the combined taxon as present when one or more of the contributing taxa was present. For example, we considered "Poplar/Tulip Poplar" present if "Poplar," "Tulip Poplar," and/or "Poplar/Tulip Poplar" was present in the original dataset.

To correspond with the reduced dimensions of the response variables, we additionally modified our calculation of effort. Specifially, we averaged the effort of all taxa at a given corner that were combined. For example, the effort of the "Poplar/Tulip Poplar" taxon was the average of the effort of any "Poplar," "Tulip Poplar," and "Poplar/Tulip Poplar" observations at a given corner.

***Data availability***

All code for processing the data and running GJAM is available from our Github repository: https://github.com/amwillson/gjam.

![Rplot02.jpeg](/Users/Aly/Downloads/Rplot02.jpeg)

Figure 1. **Number of observations (presence of taxon) per taxon before and after reducing dimensions of the response variables.**